In [24]:
import numpy as np
import pandas as pd

import os
#os.sys.path
import sys
sys.path.append('../src')

from pathlib import Path

import matplotlib.pyplot as plt
from PIL import Image
from numpy import asarray
import cv2

In [25]:
# importing tensorflow model libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten, BatchNormalization
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.metrics import categorical_accuracy
from tensorflow.keras.models import model_from_json,load_model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import *
import tensorflow.keras.backend as K
import json
import time

In [26]:
from sklearn.model_selection import train_test_split

### IMGS

In [17]:
inp1 = Path.home()/'Iron'/'inp1'

In [18]:
train_imgs = inp1/'TRAIN'
train_imgs_haar = inp1/'TRAIN_haar'
val_imgs = inp1/'VALIDATION'
test_imgs = inp1/'TEST'
demo_imgs = Path.cwd().parent/'demo'
demo_imgs_faces = Path.cwd().parent/'demo_haar'
demo_imgs_haar = Path.cwd().parent/'demo_faces'

In [19]:
demo1 = demo_imgs/'demooo_01.jpg'

In [20]:
print(df.shape, df.size, df.columns)

NameError: name 'df' is not defined

In [27]:
demo1.is_file()

True

### DATASETS to check

In [28]:
df_fer = pd.read_csv(inp1/'Fer.csv',encoding = "ISO-8859-1")
df_train = pd.read_csv(inp1/'Training_Data.csv',encoding = "ISO-8859-1")
df_test = pd.read_csv(inp1/'Testing_Data.csv',encoding = "ISO-8859-1")
df_val = pd.read_csv(inp1/'Validation_Data.csv',encoding = "ISO-8859-1")

In [29]:
face_cascade = cv2.CascadeClassifier('../src/haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('../src/haarcascade_eye.xml')
smile_cascade = cv2.CascadeClassifier('../src/haarcascade_smile.xml')

In [30]:
df = df_fer.copy()
print(df.shape, df.size, df.columns)

(35887, 3) 107661 Index(['emotion', 'pixels', 'Usage'], dtype='object')


In [31]:
df.emotion.unique()

array([0, 2, 4, 6, 3, 5, 1])

In [32]:
emos = {0:'Angry',1: 'Disgust',2:'Fear',3:'Happy',4:'Sad',5:'Surprise',6:'Neutral'}
#df['emos'] = df.emotion.map(emos)
df['emotion_names'] = df.emotion.map(emos)

In [33]:
emos2 = {0:'unhappy', 1:'unhappy',2:'unhappy',3:'happy',4:'unhappy',5:'unhappy',6:'unhappy'}

In [34]:
df['emo'] = df.emotion.map(emos2)

In [35]:
emotions = pd.get_dummies(df['emo']).to_numpy()

In [36]:
df2 = pd.get_dummies(df, columns = ['emo'])

In [37]:
df2.head()

,emotion,pixels,Usage,emotion_names,emo_happy,emo_unhappy
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training,Angry,0,1
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training,Angry,0,1
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training,Fear,0,1
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training,Sad,0,1
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training,Neutral,0,1


In [38]:
df2.Usage.value_counts()

Training       28709
PrivateTest     3589
PublicTest      3589
Name: Usage, dtype: int64

In [39]:
drpp = ['emotion', 'Usage']
df3 = df2.drop(drpp, axis=1)

In [40]:
pth1 = Path.cwd().parent.parent

In [41]:
pth1

PosixPath('/Users/cris/Iron')

In [42]:
pth2 = Path.home()/'Iron'/'data_processed'

In [43]:
pth2

PosixPath('/Users/cris/Iron/data_processed')

In [44]:
df3.to_csv(str(pth2/'df_fer_one.csv'))

In [45]:
df4 = df3.copy()

In [46]:
df4['pixar1'] = [[float(x) for x in each.split()] for each in df4['pixels']]
df4['pixar2'] = df4['pixar1'].apply(lambda x: np.asarray(x).reshape(48,48)).apply(lambda x:x.astype('float32'))

In [47]:
df4['emo_arr1'] = df4.pixar2.copy()

In [48]:
df4.head()

,pixels,emotion_names,emo_happy,emo_unhappy,pixar1,pixar2,emo_arr1
0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Angry,0,1,"[70.0, 80.0, 82.0, 72.0, 58.0, 58.0, 60.0, 63....","[[70.0, 80.0, 82.0, 72.0, 58.0, 58.0, 60.0, 63...","[[70.0, 80.0, 82.0, 72.0, 58.0, 58.0, 60.0, 63..."
1,151 150 147 155 148 133 111 140 170 174 182 15...,Angry,0,1,"[151.0, 150.0, 147.0, 155.0, 148.0, 133.0, 111...","[[151.0, 150.0, 147.0, 155.0, 148.0, 133.0, 11...","[[151.0, 150.0, 147.0, 155.0, 148.0, 133.0, 11..."
2,231 212 156 164 174 138 161 173 182 200 106 38...,Fear,0,1,"[231.0, 212.0, 156.0, 164.0, 174.0, 138.0, 161...","[[231.0, 212.0, 156.0, 164.0, 174.0, 138.0, 16...","[[231.0, 212.0, 156.0, 164.0, 174.0, 138.0, 16..."
3,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Sad,0,1,"[24.0, 32.0, 36.0, 30.0, 32.0, 23.0, 19.0, 20....","[[24.0, 32.0, 36.0, 30.0, 32.0, 23.0, 19.0, 20...","[[24.0, 32.0, 36.0, 30.0, 32.0, 23.0, 19.0, 20..."
4,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Neutral,0,1,"[4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."


In [49]:
df4['emo_arr'] = df4['emo_arr1'].apply(lambda x: np.array([[[c] for c in i] for i in x]))

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/Caskroom/miniconda/base/envs/iron/lib/python3.8/site-packages/pandas/core/series.py", line 4138, in apply
    mapped = lib.map_infer(values, f, convert=convert_dtype)
  File "pandas/_libs/lib.pyx", line 2467, in pandas._libs.lib.map_infer
  File "<ipython-input-49-766c9390d2b0>", line 1, in <lambda>
    df4['emo_arr'] = df4['emo_arr1'].apply(lambda x: np.array([[[c] for c in i] for i in x]))
  File "<ipython-input-49-766c9390d2b0>", line 1, in <listcomp>
    df4['emo_arr'] = df4['emo_arr1'].apply(lambda x: np.array([[[c] for c in i] for i in x]))
  File "<ipython-input-49-766c9390d2b0>", line 1, in <listcomp>
    df4['emo_arr'] = df4['emo_arr1'].apply(lambda x: np.array([[[c] for c in i] for i in x]))
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/Caskroom/miniconda/base/envs/iron/lib/python3.8/site-packages/IPython/core/interactiveshell.p

TypeError: object of type 'NoneType' has no len()

In [ ]:
df4.head()

In [ ]:
# df5 = df4.rename(columns={'emotion_yn_happy': 'emo_happy','emotion_yn_unhappy':'emo_unhappy'})

In [ ]:
df5 = df4.copy()

In [ ]:
df5.head(2)

In [ ]:
df5.to_csv(str(pth2/'df_fer_v3.csv'))

In [ ]:
df5.head()

In [ ]:
df5['dums'] = df5[['emo_happy','emo_unhappy']].apply(lambda x: pd.Series([x.values]), axis=1)

In [ ]:
df5.head(1)

In [ ]:
drpp = ['pixar1','pixar2']
df5.drop(columns = drpp, inplace=True)

In [ ]:
mid = df5['pixels']
df5.drop(labels=['pixels'], axis=1, inplace = True)
df5.insert(3, 'pixels', mid)
df5.head(1)

In [ ]:
df5.to_csv(str(pth2/'df_fer_three.csv'))

In [ ]:
df5.shape

In [ ]:
df5.shape

In [ ]:
X1 = (np.stack(df5['emo_arr'])) / 255.0
y1 = np.stack(df5.dums)
X1.shape, y1.shape

In [ ]:
X1_train, X1_testval, y1_train, y1_testval = train_test_split(X1,y1,test_size=0.2)

In [ ]:
X1_test, X1_val, y1_test, y1_val = train_test_split(X1_testval,y1_testval,test_size=0.5)

In [ ]:
X1_train.shape

In [ ]:
y1_train.shape

In [ ]:
X1_test.shape

In [ ]:
y1_test.shape

In [ ]:
X1_val.shape

In [ ]:
y1_val.shape

In [ ]:
X1_test.shape, X1_val.shape, y1_test.shape, y1_val.shape 

In [ ]:
X1_test, X1_val, y1_test, y1_val

In [ ]:
train_test_split(X_testval,y_testval,test_size=0.5)

In [21]:
def transfImag(path, new_path):
    
    # ROCKET
    """
    recibe carpeta, en cada foto de esa carpeta:
    lectura
    to gray
    facecascade
    por xywh en cada cara:
        array
        reshape array a 2d
        stack array 3d
        normalizar
        expand 4d
    devuelve array x cada foto para pasarselo al modelo
    """
    counter_imgs = 0
    counter_faces = 0
    
    X_ = pd.Series([], dtype='float64')
    
    for file in sorted(path.iterdir()):
        
        counter_imgs += 1

        input_img1 = cv2.imread(str(file))
        input_img2 = cv2.cvtColor(input_img1, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(input_img2, 1.25, 6)
        
        for (x,y,w,h) in faces:
            
            counter_faces += 1
            img_data1 = input_img2 [y:y+h,x:x+w]
            img_data2 = cv2.resize (img_data1,(48,48))

            img_data3 = np.stack(img_data2) 
            img_data4 = img_data2 / 255.0
            img_data5 = img_data3 / 255.0
            
            img_data6 = np.expand_dims(img_data5,axis=0).reshape(np.expand_dims(img_data5,axis=0).shape[0], 48, 48, 1)
            
            print(img_data6.shape)
            print(img_data6)
            
            img_datashow = img_data3*255
            img_show = Image.fromarray(img_datashow)
            file_to_save = file.name.replace(".",f"_face{counter_faces}.")
            img_show.save(str(new_path/file_to_save))
            
            counter_faces = 0
            
            arr_for_model = img_data6
            
            
            return arr_for_model
    
    print('YAAA!')    
    return counter_imgs
        
    

In [22]:
def base_model():
    model = Sequential()
    input_shape = (48,48,1)
    #1st convolution layer
    model.add(Conv2D(64, (5, 5), input_shape=input_shape,activation='relu', padding='same'))
    model.add(Conv2D(64, (5, 5), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))
    
    #2nd convolution layer
    model.add(Conv2D(128, (5, 5),activation='relu',padding='same'))
    model.add(Conv2D(128, (5, 5),activation='relu',padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))

    #3rd convolution layer
    model.add(Conv2D(256, (3, 3),activation='relu',padding='same'))
    model.add(Conv2D(256, (3, 3),activation='relu',padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))

    model.add(Flatten())
    model.add(Dense(128))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Dense(7))
    model.add(Activation('softmax'))

    my_optimiser = tf.keras.optimizers.Adam(
    learning_rate = 0.001, beta_1=0.9, beta_2=0.999, 
        epsilon=1e-07, amsgrad=False, name='Adam')
    
    model.compile(loss='categorical_crossentropy',
                  metrics=['accuracy'],
                  optimizer=my_optimiser)
    
    return model

In [23]:
model_1 = base_model()

model_1.fit(X1_train, y1_train, 
            validation_data=(X1_val, y1_val), 
            epochs=15,
            verbose=1, 
            batch_size=50)

model_1.summary()

NameError: name 'X1_train' is not defined

In [ ]:
impotd

In [ ]:
#transfImag(demo_imgs, demo_imgs_faces)

In [ ]:
# HAAR CASCADE CLASSIFIER
def detect_face_eyes_smile(pth,new_pth):
    """
    Extracts all .jpg files from local path, 
    calls on haar cascade classifiers (frontalface, eyes and smile) 
    and draws detection rectangles on each .jpg 
    
    Takes: local path of directory with .jpg images
    
    Returns: individual windows for .jpg files with detection rectangles for face, eyes and smile
    """

    counter_imgs = 0
    counter_faces = 0
    counter_smiles = 0
    counter_eyes = 0
    face_cascade = cv2.CascadeClassifier('../src/haarcascade_frontalface_default.xml')
    eye_cascade = cv2.CascadeClassifier('../src/haarcascade_eye.xml')
    smile_cascade = cv2.CascadeClassifier('../src/haarcascade_smile.xml')
    
    for file in sorted(pth.iterdir()):
        if file.suffix != '.jpg':
            pass
        else:
            counter_imgs += 1
            print(file.name)
            
            img = cv2.imread(str(file))
            img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            
            #plt.imshow(img)

            # FRONTAL FACE 
            
            faces = face_cascade.detectMultiScale(
                img_gray, 
                scaleFactor=1.06,                
                minNeighbors=7,
                minSize=(30, 30), 
                flags=cv2.CASCADE_SCALE_IMAGE)
            if faces is None:
                print("No Face Found")

            for (fx,fy,fw,fh) in faces:
                
                counter_faces += 1
                roi_gray = img_gray [fy:fy+fh, fx:fx+fw] # region of interest for detection
                roi_gray2 = cv2.resize (roi_gray, (48,48))
                roi_gray3 = np.stack(roi_gray2)
                roi_gray4 = roi_gray2 / 255
                
                roi_color = img[fy:fy+fh, fx:fx+fw] # region of interest for mapping rectangle
                roi_color2 = cv2.resize (roi_color, (48,48))
                roi_color3 = np.stack(roi_color2)
                roi_color4 = roi_color2 / 255
                
                cv2.rectangle(
                    img,
                    (fx,fy),
                    (fx+fw,fy+fh),
                    #(127,0,255),
                    (0,255,0),
                    2)

                # SMILES 

                smiles = smile_cascade.detectMultiScale(
                    roi_gray, 
                    scaleFactor = 1.35, 
                    minNeighbors = 8)

                for (sx, sy, sw, sh) in smiles:
                    counter_smiles += 1
                    cv2.rectangle(
                        roi_color,
                        (sx, sy),
                        (sx + sw, sy + sh),
                        #(255, 0, 130),
                        #(0,220,80),
                        (127,0,255),
                        1)

                # EYES

                eyes = eye_cascade.detectMultiScale(
                    roi_gray,
                    scaleFactor=1.05,
                    minNeighbors = 6)

                for (ex,ey,ew,eh) in eyes:
                    counter_eyes += 1
                    cv2.rectangle(
                        roi_color, 
                        (ex , ey),
                        (ex + ew, ey + eh),
                        (0,255,255),
                        1)
            
                # save images with detected regions
                file_to_save = file.name.replace(".",f"_face{counter_faces}.")
                
                #cv2.imwrite(str(pth.parent/'demo_faces'/file_to_save),img)
                cv2.imwrite(str(new_pth/file_to_save),roi_color)
                counter_imgs = 0
                counter_faces = 0
            # show the output frame
            cv2.imshow(f"img{file_to_save}", img)
            key = cv2.waitKey(100) & 0xFF
            
    cv2.destroyAllWindows(f"img{file_to_save}")
    

            
            

    
    """
        # if the `q` key was pressed, break from the loop
            if key == ord("q"):
                # do a bit of cleanup
                cv2.destroyAllWindows()
                break
        
    # do a bit of cleanup
    cv2.destroyAllWindows()
cv2.destroyAllWindows()


        """



In [ ]:
demo_imgs = Path.cwd().parent/'demo'
demo_imgs_faces = Path.cwd().parent/demo2
demo1 = demo_imgs/'A_0.jpg'
demo2 = demo_imgs/'demooo_01.jpg'

In [ ]:
face_cascade = cv2.CascadeClassifier('../src/haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('../src/haarcascade_eye.xml')
smile_cascade = cv2.CascadeClassifier('../src/haarcascade_smile.xml')

In [ ]:
cv2.imwrite(str(pth.parent/*'_haar'/file.name),roi_color)

In [ ]:
str(pth.parent/'*_haar'/file.name)